In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
import numpy as np
import random
plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kshort_pn_4GeV_Jun7_ntuple/*.root")

In [3]:
def extend_array(arr, new_attr, new_attr_name):
    members={n:arr[n] for n in arr.fields}
    members[new_attr_name] = new_attr
    return ak.zip(members)

def add_angle(arr,br=['px','py','pz','e'],name="theta"):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    arr = extend_array(arr, part.theta, name)
    return arr

def get_vector(arr):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    return part

def flat(x,axis=None): # for now must cast while waiting for coffea to catch up
    try:
        return ak.to_numpy(ak.flatten(x,axis=axis)) 
    except:
        return x

kshort_pdgid = 310
chargedpion_pdgid = 211
pion_mass = 139
kshort_mass = 497.648

In [4]:
def track_smear(kinetic_energy, theta):
    c_eres = random.gauss(1.,0.06);
    c_hres = random.gauss(0.,0.02); # 1 degrees
    return (c_eres*kinetic_energy, abs(c_hres+theta))


def test_smear(pz):
    c_eres_array = []
    for part in pz:
        for element in part:
            if element <= 50:
                c_eres = random.gauss(1,0.0404);
            elif element <= 100: 
                c_eres = random.gauss(1,0.0410);
            elif element <= 250: 
                c_eres = random.gauss(1,0.0415);
            elif element <= 500: 
                c_eres = random.gauss(1,0.0420);
            elif element <= 1000: 
                c_eres = random.gauss(1,0.0435);
            elif element <= 2000:  
                c_eres = random.gauss(1,0.0477);
            elif element <= 3000:  
                c_eres = random.gauss(1,0.0523);
            elif element <= 4000:  
                c_eres = random.gauss(1,0.0573);            
            else:    
                c_eres = random.gauss(1,0.06);
            #print("c_eres:",c_eres, 'pz:', element)
            c_eres_array=np.append(c_eres_array, [c_eres])
    #print('c_eres_array', c_eres_array)
    #print('c_eres_array shape', c_eres_array.shape)
    return (c_eres_array)

In [5]:
hists = {}
hists["Target_pion_ke"] = hist.Hist("Target SP Kaons",    
                                hist.Bin("keOG", r"Kinetic Energy [MeV]", 80, 0, 2000),
                                hist.Bin("keS", r"Energy [MeV]", 80, 0, 2000),
                            )       
hists["Target_pion_theta"] = hist.Hist("Target SP Kaons",    
                                hist.Bin("keOG", r"$\theta [^{\circ}]$", 80, 0, 4),
                                hist.Bin("keS", r"$\theta [^{\circ}]$", 80, 0, 4),
                            ) 
hists["Target_pion_mass"] = hist.Hist("Target SP Kaons",    
                                hist.Bin("keOG", r"Mass [MeV]", 80, 0, 1000),
                                hist.Bin("keS", r"Mass [MeV]", 80, 0, 1000),
                            )
hists["Target_pion_pz"] = hist.Hist("Target SP Kaons",    
                                hist.Bin("keOG", r"pz [MeV]", 80, -2000, 2000),
                                hist.Bin("keS", r"pz [MeV]", 80, -2000, 2000),
                            )   

In [6]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)

    target_kaon_dau1 = ldmx_events['TSP_Kaon_dau1']
    target_kaon_dau2 = ldmx_events['TSP_Kaon_dau2']
    
    mask_2dau1 = (target_kaon_dau1.gd==0)
    mask_2dau2 = (target_kaon_dau2.gd==0)
    kshort_mask = (abs(target_kaon_dau2.mompdgID)==kshort_pdgid)
    pidau1_mask = (abs(target_kaon_dau2.pdgID)==chargedpion_pdgid)
    pidau2_mask = (abs(target_kaon_dau2.pdgID)==chargedpion_pdgid)

 
    pi1 = get_vector(target_kaon_dau1[pidau1_mask & kshort_mask & mask_2dau2])
    pi2 = get_vector(target_kaon_dau2[pidau2_mask & kshort_mask & mask_2dau2])
    pisum = (pi1+pi2)

    
    new_ke1 = track_smear(pi1.t - pion_mass, pi1.theta)[0]
    new_ke2 = track_smear(pi2.t - pion_mass, pi2.theta)[0]
    new_theta1 = track_smear(pi1.t - pion_mass, pi1.theta)[1]
    new_theta2 = track_smear(pi2.t - pion_mass, pi2.theta)[1]
    
    new_pz1 = test_smear(pi1.pz)*flat(pi1.pz)
    new_psq1 = flat(pi1.px)**2+flat(pi1.py)**2+new_pz1**2
    new_pz2= test_smear(pi2.pz)*flat(pi2.pz)
    new_psq2 = flat(pi2.px)**2+flat(pi2.py)**2+new_pz2**2
    
    
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)
    pi1_track_smeared =  ak.zip({"x": pi1.rho*np.cos(pi1.phi)*np.sin(new_theta1),
                    "y": pi1.rho*np.sin(new_theta1)*np.sin(pi1.phi),
                    "z": pi1.rho*np.cos(new_theta1),
                    "t": new_ke1+pion_mass,
                    },
                    with_name="LorentzVector")
    pi2_track_smeared =  ak.zip({"x": pi2.rho*np.cos(pi2.phi)*np.sin(new_theta2),
                    "y": pi2.rho*np.sin(new_theta2)*np.sin(pi2.phi),
                    "z": pi2.rho*np.cos(new_theta2),
                    "t": new_ke2+pion_mass,
                    },
                    with_name="LorentzVector")
    pi_smeared_sum = (pi1_track_smeared+pi2_track_smeared)

    theta_mask_pi1 = ((40*math.pi/180) >= (pi1.theta))
    theta_mask_pi2 = ((40*math.pi/180) >= (pi2.theta))
    theta_mask_pisum = ((40*math.pi/180) >= (pisum.theta))
    theta_mask_pismear = ((40*math.pi/180) >= (pi_smeared_sum.theta))
    
    hists["Target_pion_ke"].fill(
                                 keOG=flat(pisum.t-kshort_mass),
                                 keS=flat(pi_smeared_sum.t-kshort_mass),
                                )
    hists["Target_pion_theta"].fill(
                                 keOG=flat(pisum.theta),
                                 keS=flat(pi_smeared_sum.theta),
                                )
    hists["Target_pion_mass"].fill(
                                 keOG=flat(pisum.mass),
                                 keS=flat(pi_smeared_sum.mass),
                                )
    hists["Target_pion_pz"].fill(
                                 keOG=flat(pi1.pz),
                                 keS=flat(new_pz1),
                                )
    print(flat(pisum[theta_mask_pisum].t-kshort_mass).shape)
    print(flat(pi_smeared_sum[theta_mask_pi1].t-kshort_mass).shape)
    
    return

In [7]:
nchunk = 0
for chunk in ldmx_dict:
    #if nchunk >= 1: break
    nchunk += 1
    print('process',nchunk)
    ProcessChunk(chunk, hists)

process 1


ValueError: operands could not be broadcast together with shapes (143,) (181,) 

In [ ]:
print(hists["Target_pion_ke"])
fig, ax = plt.subplots(1,3,figsize=(24,6), constrained_layout=True)
hist.plot1d(hists["Target_pion_ke"].sum('keS'),ax=ax[0],clear=False);
hist.plot1d(hists["Target_pion_ke"].sum('keOG'),ax=ax[0],clear=False);
leg = ax[0].legend([r'Measured $\pi ^\pm$', r'Smeared $\pi ^\pm$'])

hist.plot1d(hists["Target_pion_theta"].sum('keS'),ax=ax[1],clear=False);
hist.plot1d(hists["Target_pion_theta"].sum('keOG'),ax=ax[1],clear=False);
leg = ax[1].legend([r'Measured $\pi ^\pm$', r'Smeared $\pi ^\pm$'])

hist.plot1d(hists["Target_pion_mass"].sum('keS'),ax=ax[2],clear=False);
hist.plot1d(hists["Target_pion_mass"].sum('keOG'),ax=ax[2],clear=False);
leg = ax[2].legend([r'Measured $K^\pm$', r'Smeared $K^\pm$'])

In [ ]:
print(hists["Target_pion_pz"])
fig, ax = plt.subplots()
hist.plot1d(hists["Target_pion_pz"].sum('keS'),ax=ax,clear=False);
hist.plot1d(hists["Target_pion_pz"].sum('keOG'),ax=ax,clear=False);
leg = ax.legend([r'Measured pz', r'Smeared pz'])

In [ ]:
print(hists["Target_pion_mass"])
fig, ax = plt.subplots()
hist.plot1d(hists["Target_pion_mass"].sum('keS'),ax=ax,clear=False);
hist.plot1d(hists["Target_pion_mass"].sum('keOG'),ax=ax,clear=False);
leg = ax.legend([r'Measured pz', r'Smeared pz'])
leg = ax.legend([r'Measured $K^\pm$', r'Smeared $K^\pm$'])